# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import g_key

from pprint import pprint

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,acapulco,16.8634,-99.8901,24.90,57,100,0.00,MX,1679410669
1,1,jamestown,42.0970,-79.2353,7.23,41,75,6.69,US,1679410068
2,2,jacksonville,30.3322,-81.6556,18.94,68,20,7.72,US,1679410655
3,3,oksfjord,70.2396,22.3479,-1.67,86,100,4.66,NO,1679410670
4,4,cherskiy,68.7500,161.3000,-26.11,99,100,3.37,RU,1679410671


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
# city_data_df.hvplot.points("Lng","Lat",geo=True, alpha=0.2, scale=point_size, color=["r","b","k"],
#     xlim=(-180, 180), ylim=(-90, 90), tiles=True, legend="top")#, label=city)

gmaps.configure(api_key=g_key)
fig = gmaps.figure()

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Configure the map plot

locations= city_data_df[["Lat","Lng"]]
humidty= city_data_df["Humidity"].astype(float)
humidity_layer= gmaps.heatmap_layer(locations, weights=humidty)
fig.add_layer(humidity_layer)
fig

fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_ideal_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

# Drop any rows with null values
city_data_ideal_df.dropna(inplace=True)

# # Display sample data
city_data_ideal_df.head()

C:\Users\kzaho\AppData\Local\Temp\ipykernel_25404\2012581335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_ideal_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,acapulco,16.8634,-99.8901,24.90,57,100,0.00,MX,1679410669
10,10,bengkulu,-3.8004,102.2655,25.53,88,99,1.45,ID,1679410674
12,12,ponta do sol,32.6667,-17.1000,23.17,65,48,2.11,PT,1679410675
13,13,bredasdorp,-34.5322,20.0403,23.39,52,20,7.82,ZA,1679410676
14,14,rikitea,-23.1203,-134.9692,26.14,79,48,5.86,PF,1679410676


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,acapulco,MX,16.8634,-99.8901,57,
10,bengkulu,ID,-3.8004,102.2655,88,
12,ponta do sol,PT,32.6667,-17.1000,65,
13,bredasdorp,ZA,-34.5322,20.0403,52,
14,rikitea,PF,-23.1203,-134.9692,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000
params = {
    "radius": radius,
    "type": "lodging",
    "minPriceLevel": 4, # Restrict data to most expensive hotels
    "key": g_key,
    "location": "-23.1203, -134.9692"

}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make and API request using the params dictionary
hotel_r = requests.get(base_url, params).json()

pprint(hotel_r)

{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}


In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "lodging",
    "minPriceLevel": 4, # Restrict data to most expensive hotels
    "key": g_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame and add to params
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make and API request using the params dictionary
    hotel_r = requests.get(base_url, params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_r["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
acapulco - nearest hotel: No hotel found
bengkulu - nearest hotel: No hotel found
ponta do sol - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
meulaboh - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
castro - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
alofi - nearest hotel: No hotel found
mufumbwe - nearest hotel: No hotel found
conakry - nearest hotel: No hotel found
koumac - nearest hotel: No hotel found
itoman - nearest hotel: No hotel found
humaita - nearest hotel: No hotel found
mapiri - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
maningrida - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
0,acapulco,MX,16.8634,-99.8901,57,No hotel found
10,bengkulu,ID,-3.8004,102.2655,88,No hotel found
12,ponta do sol,PT,32.6667,-17.1000,65,No hotel found
13,bredasdorp,ZA,-34.5322,20.0403,52,No hotel found
14,rikitea,PF,-23.1203,-134.9692,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
#%%capture --no-display

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure()
#fig = gmaps.figure(layout=figure_layout) 

# Configure the map plot
hotel_info = hotel_df.values.tolist()

locations = hotel_df[["Lat","Lng"]].astype(float)

list_of_infoboxes = [f"Lng: {hotel[3]}\nLat: {hotel[2]}\nCity: {hotel[0]}\n\
Humidity: {hotel[4]}\nHotel Name: {hotel[5]}\nCountry: {hotel[1]}" for hotel in hotel_info]

hotel_layer = gmaps.symbol_layer(
    locations, fill_color = "rgba(0, 150, 0, 0.4)",
    stroke_color = "rgba(0, 0, 150, 0.4)", scale=2,
    hover_text = list_of_infoboxes)

# Display the map
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))